<a href="https://colab.research.google.com/github/KoNozaki/quantmacro-hw/blob/main/assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2 経済成長

労働生産性の成長率を、技術進歩(TFP)と資本深化(capital deepening)に分解する成長会計を実行するプログラムを書いて提出してください(ipynbでPythonを推奨)。

対象国は”France”, ”Germany”, ”Canada”, ”Italy”, ”Japan”, ”United Kingdom”, ”United States”。 

サンプル期間は1995年から2019年。

プログラムが最後まで走るかどうか、技術進歩(TFP)と資本深化(capital deepening)それぞれのレベルおよび寄与度について、Project2(growth accounting)にあるTable5.1のように結果を示しているかチェックします。

In [2]:
import pandas as pd

# データをStata形式から読み込みます。
pwt90 = pd.read_stata('https://www.rug.nl/ggdc/docs/pwt90.dta')

# データから必要な列を抽出します。
data = pwt90[['year', 'countrycode', 'rgdpna', 'rkna', 'pop', 'emp', 'avh', 'labsh', 'rtfpna']]

# フィルタ条件として、年度が1995年以上かつ2019年以下のデータを選択します。欠損値を削除します。
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

# 対象国を指定します。
target_countries = ["FRA", "DEU", "CAN", "ITA", "JPN", "GBR", "USA"]

# 対象国のデータを抽出します。
data = data.loc[data['countrycode'].isin(target_countries)]

# 追加変数の計算
data['alpha'] = 1 - data['labsh'] # 資本シェア割合
data['y_n'] = data['rgdpna'] / data['emp'] # 実質GDP
data['tfp_term'] = data['rtfpna'] ** (1 / (1 - data['alpha'])) 
data['cap_term'] = (data['rkna'] / data['rgdpna']) ** (data['alpha'] / (1 - data['alpha']))

# データの整形
data = data.sort_values('year').groupby('countrycode').apply(lambda x: x.assign(
    alpha=1 - x['labsh'],
    y_n_shifted=100 * x['y_n'] / x['y_n'].iloc[0],
    tfp_term_shifted=100 * x['tfp_term'] / x['tfp_term'].iloc[0],
    cap_term_shifted=100 * x['cap_term'] / x['cap_term'].iloc[0],
)).reset_index(drop=True).dropna()

# 'countrycode'でグループ化して統計量を計算
grouped_data = data.groupby('countrycode')

# 'Growth Rate'列を計算
data['g'] = (grouped_data['y_n_shifted'].diff() * 100)

# 'TFP Growth'列を計算
data['tfp_g'] = (grouped_data['tfp_term_shifted'].diff() * 100)

# 'Capital Deepening'列を計算
data['capital_deepening'] = (grouped_data['cap_term_shifted'].diff() * 100)

# 'TFP Share'列を計算
data['tfp_share'] = data['tfp_term_shifted'] / (data['tfp_term_shifted'] + data['cap_term_shifted'])

# 'Capital Share'列を計算
data['capital_share'] = data['cap_term_shifted'] / (data['tfp_term_shifted'] + data['cap_term_shifted'])

# 欠損値削除
data = data.dropna()

# 'countrycode'でグループ化して統計量を計算
summary = data.groupby('countrycode').agg({
    'g': 'mean',
    'tfp_g': 'mean',
    'capital_deepening': 'mean',
    'tfp_share': 'mean',
    'capital_share': 'mean'
})

# 列名を変更
summary = summary.rename(columns={
    'g': 'Growth Rate',
    'tfp_g': 'TFP Growth',
    'capital_deepening': 'Capital Deepening',
    'tfp_share': 'TFP Share',
    'capital_share': 'Capital Share'
})

# 結果を出力
print(summary)

# カラムの平均を計算して出力
column_means = summary.mean()
print("\nAverage:")
print(column_means)

             Growth Rate  TFP Growth  Capital Deepening  TFP Share  \
countrycode                                                          
CAN            79.997047   16.559118          69.537231   0.494033   
DEU            73.696945   80.495018         105.138397   0.483141   
FRA            88.586945   42.637955          15.293201   0.520700   
GBR           122.854286  109.131462         -59.570068   0.571221   
ITA             4.365740 -126.446777         121.346283   0.439414   
JPN            99.276138    6.295415         235.091751   0.432560   
USA           185.522858  174.879929          14.410722   0.552007   

             Capital Share  
countrycode                 
CAN               0.505967  
DEU               0.516859  
FRA               0.479300  
GBR               0.428779  
ITA               0.560586  
JPN               0.567439  
USA               0.447993  

Average:
Growth Rate          93.471420
TFP Growth           43.364590
Capital Deepening    71.606789
TFP S

<ipython-input-2-834d0f6aaef8>:25: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data = data.sort_values('year').groupby('countrycode').apply(lambda x: x.assign(
